In [1]:
import os
os.chdir("/Users/jacksonwalters/Documents/GitHub/enefit-kaggle/predict-energy-behavior-of-prosumers/")
!ls

client.csv                            gas_prices.csv
county_id_to_name_map.json            historical_weather.csv
electricity_prices.csv                public_timeseries_testing_util.py
enefit                                train.csv
example_test_files                    weather_station_to_county_mapping.csv
forecast_weather.csv


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
#helper function to convert datetime strings to integers representing a time year-month-day hour-min-sec
from datetime import datetime
def datestr_to_int(datetime_str,date_format):
    return datetime.strptime(datetime_str, date_format).timestamp()

In [4]:
#load the training data, dropping NaN's
train = pd.read_csv("train.csv").dropna()
train['datetime'] = train['datetime'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d %H:%M:%S'))

In [5]:
forecast_weather = pd.read_csv("forecast_weather.csv")
#convert strings to ints
forecast_weather['origin_datetime'] = forecast_weather['origin_datetime'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d %H:%M:%S%z'))
forecast_weather['forecast_datetime'] = forecast_weather['forecast_datetime'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d %H:%M:%S%z'))
forecast_weather['data_block_id'] -= 2

In [6]:
gas_prices = pd.read_csv("gas_prices.csv")
#convert date strings to ints
gas_prices['forecast_date'] = gas_prices['forecast_date'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d'))
gas_prices['origin_date'] = gas_prices['origin_date'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d'))
gas_prices['data_block_id'] -= 2

In [7]:
electricity_prices = pd.read_csv("electricity_prices.csv")
#convert date strings to ints
electricity_prices['forecast_date'] = electricity_prices['forecast_date'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d %H:%M:%S'))
electricity_prices['origin_date'] = electricity_prices['origin_date'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d %H:%M:%S'))
electricity_prices['data_block_id'] -= 2

In [8]:
#https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/discussion/455100

In [9]:
#merge gas prices and train.csv data
#column names differ, so use left_on and right_on
df = pd.merge(train, gas_prices, left_on=['data_block_id','datetime'], right_on=['data_block_id','origin_date'], how='left')

In [10]:
#merge train and gas_prices via left join on data_block_id
#this leaves all rows of train, but matches
df = df.merge(electricity_prices, on=['data_block_id','origin_date','forecast_date'], how='left')

In [11]:
df.dropna()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,euros_per_mwh
0,0,0,1,0.713,0,1.630469e+09,0,0,0,1.630555e+09,45.62,46.29,1.630469e+09,96.99
1,0,0,1,96.590,1,1.630469e+09,0,1,0,1.630555e+09,45.62,46.29,1.630469e+09,96.99
2,0,0,2,0.000,0,1.630469e+09,0,2,1,1.630555e+09,45.62,46.29,1.630469e+09,96.99
3,0,0,2,17.314,1,1.630469e+09,0,3,1,1.630555e+09,45.62,46.29,1.630469e+09,96.99
4,0,0,3,2.904,0,1.630469e+09,0,4,2,1.630555e+09,45.62,46.29,1.630469e+09,96.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008589,15,1,0,163.077,1,1.685333e+09,635,2009117,64,1.685419e+09,29.00,34.00,1.685333e+09,8.57
2008590,15,1,1,0.000,0,1.685333e+09,635,2009118,59,1.685419e+09,29.00,34.00,1.685333e+09,8.57
2008591,15,1,1,33.713,1,1.685333e+09,635,2009119,59,1.685419e+09,29.00,34.00,1.685333e+09,8.57
2008592,15,1,3,0.000,0,1.685333e+09,635,2009120,60,1.685419e+09,29.00,34.00,1.685333e+09,8.57


In [12]:
df = df.merge(forecast_weather, left_on=['data_block_id','forecast_date'], right_on=['data_block_id','forecast_datetime'],how='outer')

In [13]:
#drop NaN rows
df = df.dropna()

In [68]:
#drop the target column and data_block_id as they are not present in test data
X = df.drop(['target','data_block_id'], axis=1).rename(columns={'datetime': 'prediction_datetime'})

In [69]:
#set the target variable
y = df['target']

In [70]:
#verify there are no NaN's
np.any(np.isnan(y))

False

In [71]:
# load the modeling modules from sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.metrics import mean_absolute_error

In [72]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [73]:
#create a multivariable linear regression
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

LinearRegression()

In [74]:
#predict on the test data
lin_pred = regr.predict(X_test)

In [75]:
mean_absolute_error(lin_pred,y_test)

287.36847243318135

In [76]:
#train the random forest regressor
rf = RandomForestRegressor(n_estimators=1)
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=1)

In [77]:
#predict the target values using the random forest regressor
rf_pred = rf.predict(X_test)

In [78]:
#mean absolute error for random regressor
mean_absolute_error(y_test,rf_pred)

0.13657416564258448

In [79]:
#perform k-fold cross-validation
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    #train the model
    lin_regr = linear_model.LinearRegression()
    lin_regr.fit(X_train, y_train)
    #predict on the test data
    lin_pred = regr.predict(X_test)
    #evaluate the model
    print(mean_absolute_error(lin_pred,y_test))

273.6317034753345
284.87632728535175
269.9097005102265
297.629482516418
310.0367308154175


In [80]:
import pickle
# save the linear model to disk
linear_filename = '../models/linear_model.sav'
pickle.dump(regr, open(linear_filename, 'wb'))
random_forest_filename = '../models/random_forest_model.sav'
pickle.dump(rf, open(random_forest_filename, 'wb'))